In [1]:
# Import statements
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score, train_test_split

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import resample

# Models
from sklearn.linear_model import SGDClassifier # SGD model
from sklearn.ensemble import RandomForestClassifier # Random Forest model
from sklearn.neural_network import MLPClassifier # Neural network

In [1]:
# If the dataset is too sparse, try performance of these instead
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# Reading into data frame from file
data_frame = pd.read_csv('./creditcard.csv')

In [ ]:
# Profiling dataset
from ydata_profiling import ProfileReport

profile = ProfileReport(data_frame, title="Profiling Report")
profile.to_file("./profiling/data_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
# Dataset Validation
# Validation the dataset isn't malformed and list features
# data_frame.head()
# data_frame.columns

# Exploration
# data_frame.shape

In [7]:
# Data Preprocessing
# Check if data has null values (has none)
#print(data_frame.isnull().sum())
# Check if data has zero values (has some in Amount)
#print((data_frame["Class"] == 1).sum())
#print((data_frame["Class"] == 0).sum())

In [12]:
# Data to fit and targets
feature_set = ["Time", "V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "V10", "V11", "V12", "V13", "V14", "V15", "V16", "V17", "V18", "V19", "V20", "V21", "V22", "V23", "V24", "V25", "V26", "V27", "V28", "Amount"]\
# sub_set = ["Time", "V1", "V2", "V3", "V4", "Amount"]
target_set = ["Class"]

X = data_frame[feature_set]
y = data_frame[target_set]

# print(X)
# print(y)

In [12]:
# Splits data into 80-20 train-test
# Prevents data leakage (where training and test sets influence each other in scaling)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [7]:
# Scaling with each method, preserving unscaled values for comparison
standard_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

# Standard scaling
X_train_standard_scaled = standard_scaler.fit_transform(X_train)
X_test_standard_scaled = standard_scaler.transform(X_test)
# Minmax Scaling
X_train_minmax_scaled = minmax_scaler.fit_transform(X_train)
X_test_minmax_scaled = minmax_scaler.transform(X_test)

In [7]:
# Balancing data set with resampling



In [6]:
# Initialize model(s)
SGD_model = SGDClassifier(random_state=1) # Apparently prefers standard scaling
RF_model = RandomForestClassifier(random_state=1) # Theoretically needs no scaling
MLP_model = MLPClassifier(random_state=1)

In [13]:
# Does cross validation with 5 folds using our training data post split
kf = KFold(n_splits=5)
cv_scores = cross_val_score(SGD_model, X_train, y_train, cv=kf, scoring="accuracy")
print("Cross validation scores:", cv_scores)
print(f"Mean Validation Score: {np.mean(cv_scores)}")

/home/quetzl/.pyenv/versions/ml-project-2025/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/quetzl/.pyenv/versions/ml-project-2025/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/quetzl/.pyenv/versions/ml-project-2025/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/quetzl/.pyenv/versions/ml-project-2025/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: D

Cross validation scores: [0.99828831 0.9983322  0.99771775 0.99837609 0.99800303]
Mean Validation Score: 0.9981434747306283
